In [ ]:
from neo4j import GraphDatabase


class Neo4jConnection:

    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__password = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(
                self.__uri, auth=(self.__user, self.__password)
            )
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        result = None
        try:
            session = (
                self.__driver.session(database=db)
                if db is not None
                else self.__driver.session()
            )
            result = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return result

In [ ]:
def propose_sessions(uri, user, password):
    """
    Iterates through Visitor_this_year nodes with assist_year_before = "1",
    finds sessions they attended last year, and proposes 5 similar sessions
    from this year.

    Args:
        uri (str): Neo4j URI.
        user (str): Neo4j username.
        password (str): Neo4j password.
    """

    conn = Neo4jConnection(uri, user, password)

    # 1. Get all Visitor_this_year nodes with assist_year_before = "1"
    query_visitors = """
    MATCH (v_this:Visitor_this_year {assist_year_before: '1'})
    RETURN v_this
    """
    visitors = conn.query(query_visitors)

    for record in visitors:
        v_this = record["v_this"]
        badge_id_last_year = v_this.get("BadgeId_last_year")

        if badge_id_last_year:
            # 2. Find sessions attended by this visitor last year
            query_attended_sessions = """
            MATCH (v_last:Visitor_last_year {BadgeId: $badge_id_last_year})
            MATCH (v_last)-[:attended_session]->(s_last:Sessions_past_year)
            RETURN s_last
            """
            attended_sessions = conn.query(
                query_attended_sessions, {"badge_id_last_year": badge_id_last_year}
            )

            if attended_sessions:
                # 3. Extract streams from attended sessions
                last_year_streams = set()
                for session_record in attended_sessions:
                    s_last = session_record["s_last"]
                    streams = s_last.get("stream", "").split(
                        ";"
                    )  # Handle potential None
                    last_year_streams.update(stream.strip() for stream in streams)

                # 4. Find similar sessions this year
                similar_sessions = []
                for stream in last_year_streams:
                    query_similar_sessions = """
                    MATCH (s_this:Sessions_this_year)
                    WHERE $stream IN split(s_this.stream, ';')
                    RETURN s_this
                    """
                    sessions_this_year = conn.query(
                        query_similar_sessions, {"stream": stream}
                    )
                    similar_sessions.extend(sessions_this_year)

                # 5. Propose up to 5 sessions
                print(f"Visitor {v_this.get('BadgeId')}:")
                if similar_sessions:
                    print("  Sessions attended last year:")
                    for session_record in attended_sessions:
                        s_last = session_record["s_last"]
                        print(
                            f"    - {s_last.get('title')} (Stream: {s_last.get('stream')})"
                        )
                    print("  Recommended sessions this year:")
                    for i, session_record in enumerate(
                        similar_sessions[:5]
                    ):  # Limit to 5
                        s_this = session_record["s_this"]
                        print(
                            f"    {i+1}. {s_this.get('title')} (Stream: {s_this.get('stream')})"
                        )
                else:
                    print("  No similar sessions found for this visitor.")
            else:
                print(
                    f"  Visitor with BadgeId_last_year '{badge_id_last_year}' did not attend any sessions last year."
                )
        else:
            print(f"  Visitor {v_this.get('BadgeId')} does not have BadgeId_last_year.")

    conn.close()

In [ ]:
# Replace with your Neo4j credentials

uri = "bolt://127.0.0.1:7687"
user = "neo4j"
password = ""  # Replace with your password.
propose_sessions(uri, user, password)